In [ ]:
# -*- coding: utf-8 -*-
# 1.导入包以及设置随机种子
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from math import sqrt
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline

import random
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [ ]:
#定义数据集函数  
class Mydataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __getitem__(self, index):
        feature = self.features[index]
        label = self.labels[index]
        return feature, label

    def __len__(self):
        return len(self.features)
    
# 2.定义模型
class Net(nn.Module):
    def __init__(self, hidden_size):
        super(Net, self).__init__()
        self.rnn = nn.LSTM(args.dimension, 
                           hidden_size,
                           args.layers,
                           batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, inputs):
       
        s_o,_ = self.rnn(inputs)
        s_o = s_o[:, -1, :]
        x = F.dropout(F.relu(self.fc1(s_o)),0.2,training=True)
        x = F.relu(self.fc2(x))
        return torch.squeeze(x)

In [ ]:
def DC(xs, xo) -> float: #纳什效率系数
    x_mean = xo.mean()
    SST = np.sum((xo - x_mean) ** 2)
    SSRes = np.sum((xo - xs) ** 2)
    DC = 1 - (SSRes / SST)
    print("纳什效率系数为"+str(DC))
    return DC

In [ ]:
#加载模型
model = torch.load("model/lstm_mode.pth")

In [ ]:
datatest = pd.read_csv('./testdata.csv',low_memory=False)

datatest.set_index('time', inplace=True)

#rainfall=datatest.iloc[startNO, 0]
#print(rainfall)
#处理数据
data_2 = []
for i in range(len(datatest) - args.sequence_length - args.delay):
    data_2.append(datatest.iloc[i: i + args.sequence_length + args.delay])
data_2 = np.array([df.values for df in data_2])
x2 = data_2[:, :-args.delay, :]
y2 = data_2[:, -1, 0]
x2 = x2.astype(np.float32)
y2 = y2.astype(np.float32)
#归一化
mean2 = x2.mean(axis=0)
std2 = x2.std(axis=0)
#x2 = (x2 - mean2)/args.STD
x2 = (x2 - mean2)/std2
#实例化数据集
test_ds2 = Mydataset(x2, y2)
test_dl2 = torch.utils.data.DataLoader(
                                   test_ds2,
                                   batch_size=args.BTACH_SIZE
)
 
#进行预测
model.to('cuda')
predictions = []
with torch.no_grad():
    for x, y in test_dl2:
        if torch.cuda.is_available():
            x, y = x.to('cuda'), y.to('cuda')
        #print(x)
        y_pred = model(x)
        #print(y_pred)
        predictions.append(y_pred.cpu().numpy())

#预测结果
predictions = np.hstack(predictions)


In [ ]:
predictions = np.hstack(predictions)

plt.plot(y2[:],label="target")
plt.plot(predictions[:],label="predict")
plt.legend()


In [ ]:
mse = np.sum((y2 - predictions) ** 2) / len(y2)
rmse = sqrt(mse)
mae = np.sum(np.absolute(y2 - predictions)) / len(y2)
r2 = 1-mse/ np.var(y2)#均方误差/方差
print(" mae:",mae,"mse:",mse," rmse:",rmse," r2:",r2)
NH=DC(predictions,y2)

In [ ]:
np.savetxt("outputdata.txt", predictions[:],fmt='%f',delimiter=',')